In [1]:
%cd ../..

/home/eli/AnacondaProjects/surprisal_regression


In [2]:
import hydra
import math
from omegaconf import OmegaConf
import pyro
import torch
import torch.nn as nn
import torch.nn.functional as F

In [3]:
from src.models.components.linear_regression import TrialwiseLinearRegression
import src.eval as evaluation

In [4]:
with hydra.initialize(version_base="1.3", config_path="../../configs", job_name="eval"):
    cfg = hydra.compose(config_name="eval.yaml", overrides=["ckpt_path=logs/train/runs/2025-03-11_16-49-46/checkpoints/last.ckpt", "data=synthetic_surprise", "model.importance.ablations=['surprise']"],
                    return_hydra_config=True)
    hydra.core.hydra_config.HydraConfig.instance().set_config(cfg)
    _, objects = evaluation.evaluate(cfg)

/home/eli/miniforge3/envs/ephys/lib/python3.12/site-packages/lightning/pytorch/utilities/parsing.py:209: Attribute 'importance' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['importance'])`.
[rank: 0] No logger configs found! Skipping...
[striatum:2231886] shmem: mmap: an error occurred while determining whether or not /tmp/ompi.striatum.1000/jf.0/1995309056/shared_mem_cuda_pool.striatum could be created.
[striatum:2231886] create_and_attach: unable to create shared memory BTL coordinating structure :: size 134217728 
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Restoring states from the checkpoint path at logs/train/runs/2025-03-11_16-49-46/checkpoints/last.ckpt
/home/eli/miniforge3/envs/ephys/lib/python3.12/site-packages/lightning/pytorch/trainer/call.py:277: Be aware that when using `ckpt_path`, callbacks used to 

Testing: |                                                                                                    …

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│     test/log_evidence     │      -54998.1328125       │
│         test/loss         │         59382.125         │
└───────────────────────────┴───────────────────────────┘

In [5]:
objects['model'].importance.angle_alpha

tensor([0.3735, 0.6265], grad_fn=<DivBackward0>)

In [6]:
(objects['model'].importance.repetition_q_loc, objects['model'].importance.repetition_q_log_scale.exp())

(Parameter containing:
 tensor([-0.7512], requires_grad=True),
 tensor([0.0595], grad_fn=<ExpBackward0>))

In [7]:
(objects['model'].importance.selectivity_q_loc, objects['model'].importance.selectivity_q_log_scale.exp())

(Parameter containing:
 tensor([0.0023, 0.4537], requires_grad=True),
 tensor([0.1066, 0.1921], grad_fn=<ExpBackward0>))

In [8]:
log_bayes_factor = -36209.46875 - (-55140.1015625)
log_bayes_factor

18930.6328125

In [9]:
from scipy.stats import chi2

In [10]:
chi2(df=4).ppf(1 - 1e-9)

47.87945579007457

In [11]:
2 * log_bayes_factor

37861.265625